<pre>
1. Data Preprocessing
    (1) feature selection(6~7) by feature importance <- 일반적으로 6~7개를 선별함. 이유는 차후 추가        1
    (2) Time Series -> Image(Candle Stick Learner) 또는 기타 기법으로 시계열성 데이터를 이미지로 변환
    (3) Data Scaling(MinMax, StandardScaler, RobustScalcer)                                      2
    (4) Exponential Smoothing(Double, Triple)
    (5) Clustering(KMeans, T-SNE) <- 종목 분류                                                     3
2. Modeling
    (1) Simple Models
        Tradictional Models : statistical models, machine learning
        Base Model : RNN, LSTM, GRU, CNN(for ts-image data)
    (2) SOTA Models
        Reinforcement Learning, LSTM-CNN, Transformer, Kalman-Net
</pre>

# Stock Clustering
<pre>
종목 별 특성을 파악해 관련 훈련을 시키기 위해 군집방법을 사용한다.
직관적으로 유클리드 거리와 같은 표준 클러스터링 알고리즘에 사용되는 거리 측정은 시계열에 적합하지 않은 경우가 많다.
더 나은 접근방식은 기본 거리 측정을 동적 시간 왜곡과 같은 시계역 비교를 위한 메트릭으로 바꾸는 것이다.
그래서 DTW(Dynamic Time Warping Matching)을 사용한다.
- 시간, 속도 또는 길이가 정확히 일치하니 않는 두 시간 시퀀스 간의 유사성을 측정하는 기술 
</pre>

In [1]:
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [12]:
from src.data_loader import *

In [4]:
df = pd.read_csv('concat.csv', index_col=0)
df.drop(df.columns[-5:], axis = 1, inplace=True)
del_ticker_list = df['Ticker'].value_counts()[df['Ticker'].value_counts()<=252*4].index
del_ticker_index = df[df['Ticker'].isin(del_ticker_list)].index
df_new = df.drop(del_ticker_index) 

data_list = []
for stock in tqdm(df_new['Ticker'].unique()):
    _ = df_new[df_new['Ticker']==stock].set_index('Datetime').sort_index().iloc[-252*4:]['Adj Close']
    data_list.append(_)

In [40]:
concat = pd.concat(data_list, axis=1, ignore_index= True)
concat.columns = df_new['Ticker'].unique()
concat.dropna(how = 'any', inplace=True)
concat

,AACG,AADI,AAL,AAME,AAOI,AAON,AAPL,AATC,AAWW,ABCB,...,ZIONO,ZIONP,ZIVO,ZKIN,ZLAB,ZS,ZTEK,ZUMZ,ZYNE,ZYXI
Datetime,,,,,,,,,,,,,,,,,,,,,
2018-11-02,1.17,31.500000,35.653488,3.042611,20.490000,38.004932,49.972355,5.027222,55.209999,41.052216,...,20.961285,19.395184,0.14,2.500,17.240000,39.419998,0.33,22.950001,5.910,2.748664
2018-11-05,1.13,30.750000,35.151600,3.042611,19.440001,39.407829,48.553719,4.937450,54.730000,40.713963,...,20.961285,19.305046,0.14,2.500,17.030001,39.770000,0.33,22.719999,5.890,2.792432
2018-11-06,1.19,31.950001,34.610344,3.042611,20.299999,40.781506,49.078789,4.937450,54.939999,41.014633,...,21.024542,19.215158,0.15,2.400,17.219999,40.970001,0.33,22.059999,5.800,3.011275
2018-11-07,1.19,31.500000,36.381710,3.435206,19.379999,42.096722,50.567265,5.009267,57.080002,41.230743,...,20.969193,19.217592,0.13,2.490,18.299999,41.000000,0.33,21.379999,6.020,2.888723
2018-11-08,1.11,31.500000,36.273457,3.533355,21.700001,42.700741,50.390827,5.054154,56.169998,41.982452,...,21.032450,19.330248,0.14,2.455,17.860001,41.740002,0.33,21.620001,6.390,2.993768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-03,1.75,13.990000,11.920000,2.800000,2.750000,55.299999,142.449997,4.570000,97.230003,45.680000,...,25.240000,19.400000,3.01,0.838,34.279999,170.509995,1.63,21.780001,0.776,9.610000
2022-10-04,1.82,14.500000,12.950000,2.800000,2.940000,56.299999,146.100006,4.550000,97.769997,48.340000,...,25.270000,20.000000,3.14,0.820,36.599998,176.000000,1.51,22.309999,0.760,9.730000
2022-10-05,1.83,14.320000,12.870000,2.800000,3.010000,56.349998,146.399994,4.480000,97.919998,48.490002,...,25.180000,19.725000,3.14,0.808,37.070000,180.149994,1.54,22.030001,0.730,9.510000


In [44]:
mm = MinMaxScaler()
cc_df = pd.DataFrame(columns = concat.columns, data = mm.fit_transform(concat))
ddd = mm.fit_transform(concat)
# K-means를 하기 위해서는 3차원 데이터로 변경해야됨. 
# (데이터 feature, 데이터 length, 1)로
ddd = ddd.T.reshape(2360, 987,1)

In [59]:
model = TimeSeriesKMeans(n_clusters=10, metric="dtw", max_iter=10)
model.fit(ddd)

TimeSeriesKMeans(max_iter=10, metric='dtw', n_clusters=10)